<a href="https://colab.research.google.com/github/nirb28/nn_catalyst/blob/main/src/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
def resolve_path_gdrive(relativePath):
    if os.path.exists('/content/drive'):
        return '/content/drive/MyDrive/work/gdrive-workspaces/git/individual_models/' + relativePath
    else:
        from utils import get_project_root
        return get_project_root() + "/" + relativePath
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
#!pip install xlsxwriter
import xlsxwriter

In [2]:
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import xlsxwriter
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import AdamW


# Load the data
descriptors_path = 'descriptors.csv'
targets_path = 'compiled_data.csv'

In [3]:
descriptors_df = pd.read_csv(resolve_path_gdrive(descriptors_path))
targets_df = pd.read_csv(resolve_path_gdrive(targets_path))

C:\Users\dalje\AppData\Local\Temp\ipykernel_15436\1299308738.py:1: DtypeWarning: Columns (90,91,92,93,94,95,96,97,98,138,139,140,147,148,149,165,166,167,174,175,176,183,184,185,189,190,191,192,193,194,201,202,203,210,211,212,219,220,221,228,229,230,234,235,236,237,238,239,240,241,242,297,298,299,300,301,302,303,304,305,342,343,344,345,346,347,354,355,356,363,364,365,381,382,383,390,391,392,399,400,401,405,406,407,408,409,410,417,418,419,426,427,428,435,436,437,444,445,446,450,451,452,453,454,460,461,462,468,469,470,484,485,486,492,493,494,500,501,502,506,507,508,509,510,516,517,518,524,525,526,532,533,534,540,541,542,546,547,548,549,550,556,557,558,564,565,566,580,581,582,588,589,590,596,597,598,602,603,604,605,606,612,613,614,620,621,622,628,629,630,636,637,638,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,833,870,871,875,883,886,887,894,902,1351,1452) have mix

In [4]:
# Show sample rows
print("\nSample Rows from Descriptors DataFrame:")
print(descriptors_df.head())
print("\nSample Rows from Targets DataFrame:")
print(targets_df.head())


Sample Rows from Descriptors DataFrame:
   Label        ABC     ABCGG  nAcid  nBase             SpAbs_A  \
0   9268   4.719397  5.004088      0      0   6.720566232730447   
1  10488  10.334062  9.836417      0      0  16.752497538971177   
2  25579   5.875634  5.566041      0      0    9.43114762028933   
3   8952   6.611250  6.890735      1      0   10.68725972618713   
4  23681   7.249407  6.976306      0      0  11.945821561028193   

              SpMax_A           SpDiam_A              SpAD_A  \
0  2.1010029896154583  4.202005979230917   6.720566232730447   
1  2.3623398328574394  4.724679665714879  16.752497538971177   
2  2.1753277471610764  4.350655494322151    9.43114762028933   
3    2.28774942353935  4.425414875225794   10.68725972618713   
4  2.2671838628844996     4.534367725769  11.945821561028193   

              SpMAD_A  ...     SRW10     TSRW10          MW        AMW WPath  \
0  0.9600808903900638  ...  8.123558  33.343946  136.047505   6.802375    46   
1  1.196606

In [5]:
descriptors_df.shape

(28731, 1827)

In [5]:
# Keep only numeric columns
descriptors_numeric = descriptors_df.select_dtypes(include=['number'])
targets_numeric = targets_df.select_dtypes(include=['number'])

# Merge the numeric dataframes on the common label column
numeric_data = pd.merge(descriptors_numeric, targets_numeric, left_on='Label', right_on='mol_num')
numeric_data = numeric_data.drop(columns=['Label', 'mol_num'])

number_of_target_cols = len(targets_numeric.columns) - 1

In [11]:

# Separate features and targets
X = numeric_data.iloc[:, :-number_of_target_cols]  # Assuming the last 30 columns are targets
y = numeric_data.iloc[:, -number_of_target_cols:]

# Apply variance threshold
selector = VarianceThreshold()
X_high_variance = selector.fit_transform(X)

# Convert to numpy arrays
X = X_high_variance
y = y.values

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Standardize the data
scaler_X = StandardScaler().fit(X_train)
scaler_y = StandardScaler().fit(y_train)

X_train = scaler_X.transform(X_train)
X_val = scaler_X.transform(X_val)
X_test = scaler_X.transform(X_test)

y_train = scaler_y.transform(y_train)
y_val = scaler_y.transform(y_val)
y_test = scaler_y.transform(y_test)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32, device=device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32, device=device)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32, device=device)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32, device=device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32, device=device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32, device=device)


In [12]:
# Create DataLoader for batch processing
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# Define the individual model class
class SingleTargetNet(nn.Module):
    def __init__(self, input_size, dropout_rate=0.5):
        super(SingleTargetNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 1)
        self.fc_skip = nn.Linear(1024, 512)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x1 = F.relu(self.bn1(self.fc1(x)))
        x1 = self.dropout(x1)

        x2 = F.relu(self.bn2(self.fc2(x1)))
        x2 = self.dropout(x2)

        # Skip connection
        x2 += self.fc_skip(x1)

        x3 = self.fc3(x2)
        return x3

# Function to train and evaluate individual models
def train_and_evaluate(target_index):
    model = SingleTargetNet(X_train.shape[1])
    model.to(device=device)
    criterion = nn.MSELoss()
    optimizer = AdamW(model.parameters(), lr=0.001)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)

    best_val_loss = np.inf
    patience_counter = 0
    num_epochs = 150

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, targets[:, target_index])
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Target {target_index} - Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs).squeeze()
                loss = criterion(outputs, targets[:, target_index])
                val_loss += loss.item()
        val_loss /= len(val_loader)
        print(f'Target {target_index} - Validation Loss: {val_loss}')

        scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), f'best_model_target_{target_index}.pth')
        else:
            patience_counter += 1
            if patience_counter >= 15:
                print(f'Target {target_index} - Early stopping triggered')
                break

    model.load_state_dict(torch.load(f'best_model_target_{target_index}.pth'))

    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, targets[:, target_index])
            test_loss += loss.item()
    test_loss /= len(test_loader)
    print(f'Target {target_index} - Test Loss: {test_loss}')

    return model, test_loss


In [13]:

# Train and evaluate individual models for each target
test_losses = []
models = []
for target_index in range(y_train.shape[1]):
    model, test_loss = train_and_evaluate(target_index)
    models.append(model)
    test_losses.append(test_loss)

# Print average test loss
avg_test_loss = np.mean(test_losses)
print(f'Average Test Loss for Individual Models: {avg_test_loss}')

# Initialize Excel writer
output_path = 'individual_model_predictions_with_plots.xlsx'
writer = pd.ExcelWriter(output_path, engine='xlsxwriter')
workbook = writer.book

# Create necessary Excel sheets
train_sheet = workbook.add_worksheet('Train')
val_sheet = workbook.add_worksheet('Validation')
test_sheet = workbook.add_worksheet('Test')

# Prepare DataFrames for train, validation, and test predictions
train_df = pd.DataFrame()
val_df = pd.DataFrame()
test_df = pd.DataFrame()

r2_scores, rmse_scores, mae_scores = [], [], []

def create_excel_chart(sheet_name, target_index, worksheet, df, start_row, start_col):
    chart = workbook.add_chart({'type': 'scatter'})

    observed_col = f'Observed_{target_index}'
    predicted_col = f'Predicted_{target_index}'

    chart.add_series({
        'name': f'Target {target_index}',
        'categories': [sheet_name, start_row+1, df.columns.get_loc(observed_col), start_row+df.shape[0], df.columns.get_loc(observed_col)],
        'values': [sheet_name, start_row+1, df.columns.get_loc(predicted_col), start_row+df.shape[0], df.columns.get_loc(predicted_col)],
        'marker': {'type': 'circle', 'size': 5},
        'trendline': {
            'type': 'linear',
            'display_equation': True,
            'display_r_squared': True,
        }
    })
    chart.set_title({'name': f'Parity Plot for Target {target_index}'})
    chart.set_x_axis({'name': 'Observed'})
    chart.set_y_axis({'name': 'Predicted'})
    chart.set_legend({'none': True})

    # Make axes square with the same unit ranges on x and y axis
    min_val = min(df[observed_col].min(), df[predicted_col].min())
    max_val = max(df[observed_col].max(), df[predicted_col].max())
    chart.set_x_axis({'min': min_val, 'max': max_val})
    chart.set_y_axis({'min': min_val, 'max': max_val})

    worksheet.insert_chart(start_row + df.shape[0] + 2, start_col, chart)

    # Calculate metrics
    observed = df[observed_col]
    predicted = df[predicted_col]
    r2 = r2_score(observed, predicted)
    rmse = mean_squared_error(observed, predicted, squared=False)
    mae = mean_absolute_error(observed, predicted)

    # Write metrics to Excel
    metrics_start_row = start_row + df.shape[0] + 22
    worksheet.write(metrics_start_row, start_col, f'Target {target_index}')
    worksheet.write(metrics_start_row + 1, start_col + 1, 'R²')
    worksheet.write(metrics_start_row + 1, start_col + 2, r2)
    worksheet.write(metrics_start_row + 2, start_col + 1, 'RMSE')
    worksheet.write(metrics_start_row + 2, start_col + 2, rmse)
    worksheet.write(metrics_start_row + 3, start_col + 1, 'MAE')
    worksheet.write(metrics_start_row + 3, start_col + 2, mae)

    return r2, rmse, mae

for target_index in range(y_train.shape[1]):
    model = models[target_index]

    # Make predictions on the train, validation, and test sets
    model.eval()
    with torch.no_grad():
        y_train_pred = model(X_train_tensor).numpy()
        y_val_pred = model(X_val_tensor).numpy()
        y_test_pred = model(X_test_tensor).numpy()

    # Inverse transform the predictions and targets to their original scale
    y_train_pred_orig = scaler_y.inverse_transform(np.concatenate([np.zeros((y_train_pred.shape[0], target_index)), y_train_pred, np.zeros((y_train_pred.shape[0], y_train.shape[1] - target_index - 1))], axis=1))[:, target_index]
    y_val_pred_orig = scaler_y.inverse_transform(np.concatenate([np.zeros((y_val_pred.shape[0], target_index)), y_val_pred, np.zeros((y_val_pred.shape[0], y_val.shape[1] - target_index - 1))], axis=1))[:, target_index]
    y_test_pred_orig = scaler_y.inverse_transform(np.concatenate([np.zeros((y_test_pred.shape[0], target_index)), y_test_pred, np.zeros((y_test_pred.shape[0], y_test.shape[1] - target_index - 1))], axis=1))[:, target_index]

    y_train_orig = scaler_y.inverse_transform(y_train)[:, target_index]
    y_val_orig = scaler_y.inverse_transform(y_val)[:, target_index]
    y_test_orig = scaler_y.inverse_transform(y_test)[:, target_index]

    # Create dataframes for the predictions and actual values
    train_df[f'Observed_{target_index}'] = y_train_orig
    train_df[f'Predicted_{target_index}'] = y_train_pred_orig

    val_df[f'Observed_{target_index}'] = y_val_orig
    val_df[f'Predicted_{target_index}'] = y_val_pred_orig

    test_df[f'Observed_{target_index}'] = y_test_orig
    test_df[f'Predicted_{target_index}'] = y_test_pred_orig

    # Create and insert parity plots for train, validation, and test sets
    r2, rmse, mae = create_excel_chart('Train', target_index, train_sheet, train_df, start_row=0, start_col=target_index*9)
    r2_scores.append(r2)
    rmse_scores.append(rmse)
    mae_scores.append(mae)
    create_excel_chart('Validation', target_index, val_sheet, val_df, start_row=0, start_col=target_index*9)
    create_excel_chart('Test', target_index, test_sheet, test_df, start_row=0, start_col=target_index*9)

# Write dataframes to Excel sheets
train_df.to_excel(writer, sheet_name='Train', index=False)
val_df.to_excel(writer, sheet_name='Validation', index=False)
test_df.to_excel(writer, sheet_name='Test', index=False)

# Save and close the Excel file
writer.save()

# Calculate and print the average R², RMSE, and MAE for the validation set
avg_r2 = np.mean(r2_scores)
avg_rmse = np.mean(rmse_scores)
avg_mae = np.mean(mae_scores)

print(f"Average R² for Validation Set: {avg_r2}")
print(f"Average RMSE for Validation Set: {avg_rmse}")
print(f"Average MAE for Validation Set: {avg_mae}")

print(f"Predictions and plots written to {output_path}")


d:\work\conda\nn_310\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Target 0 - Epoch 1/150, Loss: 0.40643509364936775
Target 0 - Validation Loss: 0.20697216335229757
Target 0 - Epoch 2/150, Loss: 0.24164172206301152
Target 0 - Validation Loss: 0.191017315972869
Target 0 - Epoch 3/150, Loss: 0.23762599687750746
Target 0 - Validation Loss: 0.1520367775384973
Target 0 - Epoch 4/150, Loss: 0.23863889725606252
Target 0 - Validation Loss: 0.1499833580015636
Target 0 - Epoch 5/150, Loss: 0.22097503169008145
Target 0 - Validation Loss: 0.13777288199379678
Target 0 - Epoch 6/150, Loss: 0.21135828502067341
Target 0 - Validation Loss: 0.18949397189951525
Target 0 - Epoch 7/150, Loss: 0.20694956909183684
Target 0 - Validation Loss: 0.1217359411189469
Target 0 - Epoch 8/150, Loss: 0.1988589229834516
Target 0 - Validation Loss: 0.13973895805638012
Target 0 - Epoch 9/150, Loss: 0.19302540833539353
Target 0 - Validation Loss: 0.1428409808474343
Target 0 - Epoch 10/150, Loss: 0.19309857096409472
Target 0 - Validation Loss: 0.14131326691769971
Target 0 - Epoch 11/150, L

C:\Users\dalje\AppData\Local\Temp\ipykernel_11768\3809218730.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'best_model_target_{targe

Target 1 - Epoch 1/150, Loss: 0.3772565507011988
Target 1 - Validation Loss: 0.1449491857301171
Target 1 - Epoch 2/150, Loss: 0.20343309396696163
Target 1 - Validation Loss: 0.1530219754431306
Target 1 - Epoch 3/150, Loss: 0.1945683579585265
Target 1 - Validation Loss: 0.13039711903689838
Target 1 - Epoch 4/150, Loss: 0.18285325108232295
Target 1 - Validation Loss: 0.11854446615751196
Target 1 - Epoch 5/150, Loss: 0.1771027706222745
Target 1 - Validation Loss: 0.11296205299838287
Target 1 - Epoch 6/150, Loss: 0.17087910527068123
Target 1 - Validation Loss: 0.12178076217632468
Target 1 - Epoch 7/150, Loss: 0.1725277225714086
Target 1 - Validation Loss: 0.17971684783697128
Target 1 - Epoch 8/150, Loss: 0.16090217173644683
Target 1 - Validation Loss: 0.10861539495427434
Target 1 - Epoch 9/150, Loss: 0.15420168960217115
Target 1 - Validation Loss: 0.15171757176881884
Target 1 - Epoch 10/150, Loss: 0.1524687427345936
Target 1 - Validation Loss: 0.10377699050416307
Target 1 - Epoch 11/150, L

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.